In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib

In [ ]:
# url="http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_1"


In [363]:
def getInfo(url,grp_name,city):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    region = grp_name
#     region='муниципальный'
    str_url = checkEnd(url)
    
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]

    def getTable(table_rows):
        l = []
        for tr in table_rows:
            try:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
            except:
                pass
        #
        return l
    #
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    #####

    all_tables = bs.find_all("table")
    # l2 = getTable(all_tables[2])
    # l3 = getTable(all_tables[3])
    # l5 = getTable(all_tables[5])
    # l6 = getTable(all_tables[6])
    # l7 = getTable(all_tables[7])
    # l8 = getTable(all_tables[8])

    #######
    l2,l3,l4,l5,l6,l7,l8 = [],[],[],[],[],[],[]
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if 'Содержание'in l[0][0]:
            l2 = l
        elif ('направление'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if ('прям' in l[0][0]) | ('прям' in tmp):
                if '№п' not in tmp:
                    l3 = l
                else:
                    l5 = l
            elif ('обратн' in l[0][0]) | ('обратн' in tmp):
                if '№п' in tmp:
                    l6 = l
            #
        elif ('обратное'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if '№п' in tmp:
                l6 = l
        elif 'работы'in l[0][0]:
            l7 = l
        else:
            l8 = l
    # 
    #######
    
    
    ###
    # если кольцевой!!!
#     if ['обратное направление\n'] not in l6:
#         l8 = l7
#         l7 = l6
    ###

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")
    rt_nm = str1.split("\n")[1][2:].split("маршрут ")[1]
#     rt_nm = str1.split("\n")[1][2:].split("№")[1]
    rt_nm = rt_nm.split('"', 1)[1][:-1]

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1]
    try:
        route_bckw = l3[1][1]
    except:
        route_bckw = 'кольцевой'
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    try:
        stops_bckw = stops(l6)
    except:
        stops_bckw = 'кольцевой'
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 
    
    lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
#                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    
    def foo(nlst,i):
        return nlst[i][1]

    for i in range(9):
        try: 
            tmp = foo(nlst,i)
        except:
            tmp = None
        lst_all.append(tmp)
    # 
    lst_all.append(url)


#     work_time = nlst[0][1]
#     intervs = nlst[1][1]
#     amnt_mch = nlst[2][1]
#     duration = nlst[3][1]
#     length = nlst[4][1]
#     orgnztn = nlst[5][1]
#     podv_sost = nlst[6][1]
#     payment = nlst[7][1]
#     place_rest = nlst[8][1]
#     ####


#     # get all to list
#     lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,
#                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
#     #
    return lst_all

In [376]:
def getInfo2(url,grp_name,city):
    # Сызрань спешл
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    region = grp_name
#     region='муниципальный'
    str_url = checkEnd(url)
    
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]

    def getTable(table_rows):
        l = []
        for tr in table_rows:
            try:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
            except:
                pass
        #
        return l
    #
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    #####

    all_tables = bs.find_all("table")
    
        # 3 - наименование маршрута
    rt_nm = str(bs.find_all('b')[2]).replace("<b>","").replace("</b>","")

    def getLines(lst_s,nmb):
        lst_ok = []

        for i in range(len(lst_s)):
            lst1 = lst_s[i][nmb].split("\n")
            for s in lst1:
                if s != '':
                    lst_ok.append(s)
        # 
        return lst_ok

    def stops(lst_s,nmb):
        lst_ok = getLines(lst_s,nmb)

        str1=''
        for row in lst_ok:
            if row != '\n':
                row = row.replace("\n", "")
                str1 = str1 + "_" + row
        # 
        str1 = str1[1:]
        return str1

    ####
    
    i=0
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if "в сторону" in l[0][0]:
            l5 = getTable(all_tables[i+2])
            l6 = getTable(all_tables[i+3])
            break
        else:
            l5 = getTable(all_tables[5])
            l6 = getTable(all_tables[6])
    # 

    ####
    # 4 -маршрут следования

    route_frw = stops(l5,0)
    try:
        route_bckw = stops(l6,0)
    except:
        route_bckw = 'кольцевой'
    ####

    # 5,6 - список оп
    stops_frw = stops(l5,1)
    try:
        stops_bckw = stops(l6,1)
    except:
        stops_bckw = 'кольцевой'
    ####
    
    nlst = []
    
    lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
#                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    
    def foo(nlst,i):
        return nlst[i][1]

    for i in range(9):
        try: 
            tmp = foo(nlst,i)
        except:
            tmp = None
        lst_all.append(tmp)
    # 
    lst_all.append(url)
    
    return lst_all

In [2]:
def checkGrpNm(tmp_grp_nm):
    if tmp_grp_nm == 'маршруты':
        grp_name = 'муниципальный'
    elif  tmp_grp_nm == 'коммерческий_маршруты':
        grp_name = 'коммерческий'
    elif  tmp_grp_nm == 'пригородный_расписания':
        grp_name = 'пригородный'
    elif  tmp_grp_nm == 'сезонный_маршруты':
        grp_name = 'сезонный'
    elif  tmp_grp_nm == 'междугородный_расписания':
        grp_name = 'междугородний'
    else:
        grp_name = tmp_grp_nm
    # 
    return grp_name

In [3]:
def checkEnd(url):
    the_string = url.rsplit('php/',1)[1]
    srch = re.search('[а-яА-Я]', the_string) 
    if srch == None:
        str_url = urllib.parse.unquote(the_string)
    else:
        str_url = the_string
    #
    return str_url

In [4]:
# 
def linksCity(city):

    url_grp = 'http://www.samaratrans.info/wiki/index.php/' + city
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('a')
    links=[]
    for link in stfa:
        try:
            if 'маршрут' in link.get("title"):
                lsh = 'http://www.samaratrans.info' + link.get("href")
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [5]:
# def linksCity(city):
#     lst_links_city = []
#     lst_types = ['автобус', 'троллейбус', 'трамвай']
#     lst_route_type = ['маршруты','коммерческий_маршруты', 
#                   'пригородный_расписания','сезонный_маршруты','междугородный_расписания']
#     #
#     city_link = 'http://www.samaratrans.info/wiki/index.php/' + city
#     i=0
#     for i in range(len(lst_types)):
#         tp_link = city_link + '_' + lst_types[i] + '_' + lst_route_type[0]
#         lst_links_city.append(tp_link)
#     #
#     bus_link = city_link + '_' + lst_types[0]
    
#     i=0
#     for i in range(1,len(lst_route_type)):
#         link = bus_link + '_' + lst_route_type[i]
#         lst_links_city.append(link)
#     #
#     return lst_links_city

In [6]:
def getTable(table_rows):
    l = []
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
    #
    return l

In [21]:
def routeLinks(url_grp):
    links=[]
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('table')
    
    cnt=2
    for i in range(len(stfa)):
        table_rows = stfa[i]
        l = getTable(table_rows)
        if 'Наименование маршрута\n' in l[0]:
            cnt = i
            break
    # 
    lst_find = stfa[cnt].find_all('a')
    for link in lst_find:
        try:
            lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
            lnk_hr = link.get("href")
            ch = checkEnd(lnk_hr)
            if not any((c in ch) for c in lst_notok):
                lsh = 'http://www.samaratrans.info' + lnk_hr
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [349]:
def getCityData(city):
    lst_links = []
    lst_links = linksCity(city)
    all_routes=[]
    for grp in range(len(lst_links)):
        tmp_grp_nm = checkEnd(lst_links[grp]).split("_",2)[-1]
        grp_name = checkGrpNm(tmp_grp_nm)
        
        try:
            lst_group = []
            lst_group = routeLinks(lst_links[grp])
        except:
            lst_group = []
        for rtl in range(len(lst_group)):
            try:
                tmp_lnk_nm = checkEnd(lst_group[rtl])
                lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
                if not any((c in tmp_lnk_nm) for c in lst_notok):
                    try:
                        one_route = getInfo(lst_group[rtl],grp_name,city)
                    except:
                        one_route = getInfo2(lst_group[rtl],grp_name,city)
                    all_routes.append(one_route)
                #
            except:
                one_route = [lst_group[rtl]] * 19
                if one_route not in all_routes:
                    all_routes.append(one_route)
                #
    return all_routes
# 

In [350]:
lst_ct = ['Самара', 'Тольятти', 'Сызрань', 'Новокуйбышевск', 
          'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']
# lst_ct = ['Самара', 'Тольятти', 'Новокуйбышевск', 
#           'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']

In [377]:
lst_big = []
for i in range(len(lst_ct)):
    city = lst_ct[i]
    print(city)
    lst = getCityData(city)
#     print(len(lst_big))
    lst_big = lst_big + lst
#     print(len(lst_big))
# 

Самара
Тольятти
Сызрань
Новокуйбышевск
Чапаевск
Жигулевск
Отрадный
Кинель
Похвистнево
Безенчук


In [ ]:
# import time

In [207]:
# %time lst_big = getCityData(lst_ct[4])

In [352]:
clmn = []
for i in range(1,len(lst_big[0])+1):
    str1 = str(i)+'str'
    clmn.append(str1)
# 

In [353]:
# df_cp = df_big.copy()

In [378]:
df_big = pd.DataFrame(data=lst_big,columns=clmn)

In [379]:
print(len(df_big))

447


In [380]:
lst_not_ok = list(set(list(df_big[df_big["1str"].str.contains("www")]["1str"])))

In [381]:
lst2 = list(set(lst_not_ok))

In [382]:
print(len(lst2))

0


In [359]:
for i in lst2:
    print(i)

http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_7
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_6
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_8
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_116
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_3
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_35
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_2
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%

In [360]:
url = lst2[0]
grp_name = 'error'
city = 'Сызрань'
print(url)

http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_7


In [362]:
# ltp = getInfo2(url,grp_name,city)
ltp=getInfo2(url,grp_name,city)

IndexError: list index out of range

In [346]:
ltp

['Сызрань',
 'автобус',
 'error',
 '30',
 'Железнодорожный вокзал / ст. Сызрань-1 - п. Турбостроителей',
 ' ул. Московская _ ул. Фридриха Энгельса _ пер. Волжский _ ул. Ульяновская _ ул. Декабристов _ ул. 2-я студенческая _ ул. Гидротурбинная _ Ульяновское шоссе _ ул. Локомобильная',
 ' ул. Локомобильная _ Ульяновское шоссе _ ул. Гидротурбинная _ ул. 2-я студенческая _ ул. Декабристов _ ул. Ульяновская _ пер. Волжский _ ул. Фридриха Энгельса _ ул. Московская ',
 '1. Железнодорожный вокзал / ст. Сызрань-1 (посадка)_2. ТВ "Канон"_3. МП "Зеленое хозяйство"_4. Школа №15_5. ул. Дачная_6. ул. Студенческая_7. ДК "Авангард"_8. Военный городок_9. 2-я проходная АО "Тяжмаш"_10. п. Турбостроителей_11. ОАО "Сызраньмолоко" (высадка)',
 '1. ОАО "Сызраньмолоко" (посадка)_2. п. Турбостроителей_3. 2-я проходная АО "Тяжмаш"_4. Военный городок_5. ДК "Авангард"_6. ул. Студенческая_7. ул. Дачная_8. Школа №15_9. МП "Зеленое хозяйство"_10. ТВ "Канон"_11. Железнодорожный вокзал / ст. Сызрань-1 (высадка)',
 Non

In [374]:
# def getInfo2(url,grp_name,city):
# Сызрань спешл
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

region = grp_name
#     region='муниципальный'
str_url = checkEnd(url)

lst = str_url.split("_")
# 2 - номер маршрута
nmbr = lst[-1]
# 1 - тип транспорта
tp_ts = lst[-2]
# rt_nmbr = url.rsplit("_",1)[1]

def getTable(table_rows):
    l = []
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
    #
    return l
#
# 2 - содержание
# 3 - маршрут следования
# 5 - оп в прямом направлении
# 6 - оп в обратном направлении
# 7 - особенности работы
# 8 - расписание (две версии, для регионов!!)

#####

all_tables = bs.find_all("table")

    # 3 - наименование маршрута
rt_nm = str(bs.find_all('b')[2]).replace("<b>","").replace("</b>","")

def getLines(lst_s,nmb):
    lst_ok = []

    for i in range(len(lst_s)):
        lst1 = lst_s[i][nmb].split("\n")
        for s in lst1:
            if s != '':
                lst_ok.append(s)
    # 
    return lst_ok

def stops(lst_s,nmb):
    lst_ok = getLines(lst_s,nmb)

    str1=''
    for row in lst_ok:
        if row != '\n':
            row = row.replace("\n", "")
            str1 = str1 + "_" + row
    # 
    str1 = str1[1:]
    return str1

####
i=0
for i in range(len(all_tables)):
    l = getTable(all_tables[i])
    if "в сторону" in l[0][0]:
        l5 = getTable(all_tables[i+2])
        l6 = getTable(all_tables[i+3])
        break
    else:
        l5 = getTable(all_tables[5])
        l6 = getTable(all_tables[6])
# 

# l5 = getTable(all_tables[5])
# l6 = getTable(all_tables[6])

####
# 4 -маршрут следования

route_frw = stops(l5,0)
try:
    route_bckw = stops(l6,0)
except:
    route_bckw = 'кольцевой'
####

# 5,6 - список оп
stops_frw = stops(l5,1)
try:
    stops_bckw = stops(l6,1)
except:
    stops_bckw = 'кольцевой'
####

nlst = []

lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
#                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]

def foo(nlst,i):
    return nlst[i][1]

for i in range(9):
    try: 
        tmp = foo(nlst,i)
    except:
        tmp = None
    lst_all.append(tmp)
# 
lst_all.append(url)

#     return lst_all